In [ ]:
#imports
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from helpers.datasetHelper import get_samples, split_healthy_data
from pyswarm import pso

from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

from sklearn.metrics import make_scorer
from sklearn.metrics import cohen_kappa_score

from models import MyXGboost

In [149]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import plotly.express as px

def display_confusion_matrix_pink_variants(estimator, X_test, y_test):
    # Get predictions
    y_pred = estimator.predict(X_test)
    
    # Create confusion matrix (normalized by true labels)
    cm = confusion_matrix(y_test, y_pred, normalize='true')
    
    # Convert to percentage
    cm = cm * 100
    
    # Create a dataframe for plotting
    classes = ['HEALTHY', 'PRE-BRCA', 'BRCA']
    df_cm = pd.DataFrame(cm, index=classes, columns=classes)
    
    # Create heatmap using plotly express
    fig = px.imshow(df_cm,
                    labels=dict(x="Predicted", y="True", color="Percentage"),
                    x=classes,
                    y=classes,
                    color_continuous_scale='RdPu',
                    aspect='auto',
                    title='Confusion Matrix (Normalized by True Labels)',
                    text_auto=True)  # This adds automatic text annotations
    
    # Customize the text format to show percentages
    fig.update_traces(texttemplate="%{z:.1f}%", textfont_size=12)
    
    # Update layout for better appearance
    fig.update_layout(
        width=500,
        height=400,
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        font=dict(size=12)
    )
    
    # Ensure text is readable on all backgrounds
    fig.update_traces(textfont_color="white")
    
    return fig  

In [ ]:
def run_pso_with_progress(X, Y, estimator, n_features,
                          swarmsize=50, maxiter=10, threshold=0.7):
    lb = [0]*n_features
    ub = [1]*n_features
    progress = []

    # Create a scorer for Cohen's Kappa
    kappa_scorer = make_scorer(cohen_kappa_score)

    def objective_with_progress(weights, est, X_, Y_):
        Xw = X_ * weights
                
        # Use it in cross_val_score
        fit = 1 - cross_val_score(est, Xw, Y_, cv=5, scoring=kappa_scorer).mean()
        progress.append(fit)
        
        if len(progress) % 10 == 0:
            print(f"Eval {len(progress)}: best fitness so far = {min(progress):.4f}")
        return fit

    best_pos, best_fit = pso(
        objective_with_progress,
        lb, ub,
        args=(estimator, X, Y),
        swarmsize=swarmsize,
        maxiter=maxiter
    )

    mask = best_pos > threshold
    selected_features = np.where(mask)[0].tolist()
    return best_pos, best_fit, progress, selected_features

In [137]:
def run_two_stage_pso(X, Y, estimator, n_features,
                      # Stage 1 parameters
                      stage1_swarmsize=30, stage1_maxiter=25, stage1_threshold=0.4,
                      # Stage 2 parameters  
                      stage2_swarmsize=25, stage2_maxiter=25, stage2_threshold=0.7,
                      # General parameters
                      verbose=True):
    if verbose:
        print("="*60)
        print("STARTING TWO-STAGE PSO FEATURE SELECTION")
        print("="*60)
    
    # STAGE 1: Coarse feature selection
    if verbose:
        print(f"\nSTAGE 1: Coarse Selection")
        print(f"- Original features: {n_features}")
        print(f"- Swarm size: {stage1_swarmsize}")
        print(f"- Max iterations: {stage1_maxiter}")
        print(f"- Threshold: {stage1_threshold}")
        print(f"shape of X: {X.shape}")
        print(f"shape of Y: {Y.shape}")
        print("-" * 40)
    
    stage1_weights, stage1_fitness, stage1_progress, stage1_features = run_pso_with_progress(
        X, Y, estimator, n_features,
        swarmsize=stage1_swarmsize,
        maxiter=stage1_maxiter,
        threshold=stage1_threshold
    )
    
    if verbose:
        print(f"\nSTAGE 1 RESULTS:")
        print(f"- Features selected: {len(stage1_features)}")
        print(f"- Best fitness: {stage1_fitness:.4f}")
        print(f"- Feature reduction: {n_features} → {len(stage1_features)} "
              f"({len(stage1_features)/n_features*100:.1f}%)")
    
    # Check if stage 1 selected any features
    if len(stage1_features) == 0:
        print("WARNING: Stage 1 selected no features! Lowering threshold...")
        # Retry with lower threshold
        stage1_weights, stage1_fitness, stage1_progress, stage1_features = run_pso_with_progress(
            X, Y, estimator, n_features,
            swarmsize=stage1_swarmsize,
            maxiter=stage1_maxiter,
            threshold=stage1_threshold * 0.7  # Lower threshold
        )
        
        if len(stage1_features) == 0:
            raise ValueError("No features selected even with lowered threshold!")
    
    # STAGE 2: Refined selection on reduced feature set
    if verbose:
        print(f"\nSTAGE 2: Refined Selection")
        print(f"- Swarm size: {stage2_swarmsize}")
        print(f"- Max iterations: {stage2_maxiter}")
        print(f"- Threshold: {stage2_threshold}")
        print("-" * 40)
    
    # Create reduced dataset with only stage 1 selected features
    print(f"Original feature set shape: {X.shape}")
    print(f"Selected features from stage 1: {len(stage1_features)}")

    max_idx = max(stage1_features)
    if max_idx >= X.shape[1]:
        print(f"Index {max_idx} is too large for {X.shape[1]} features")
    
    X_reduced = X.iloc[:, np.array(stage1_features)].values
    print(f"Reduced feature set shape: {X_reduced}")
    
    # Run PSO on reduced feature set
    stage2_weights, stage2_fitness, stage2_progress, stage2_features_reduced = run_pso_with_progress(
        X_reduced, Y, estimator, len(stage1_features),
        swarmsize=stage2_swarmsize,
        maxiter=stage2_maxiter,
        threshold=stage2_threshold
    )
    
    # Map stage 2 features back to original feature indices
    final_features = [stage1_features[i] for i in stage2_features_reduced]
    
    if verbose:
        print(f"\nSTAGE 2 RESULTS:")
        print(f"- Features selected: {len(final_features)}")
        print(f"- Best fitness: {stage2_fitness:.4f}")
        print(f"- Feature reduction: {len(stage1_features)} → {len(final_features)} "
              f"({len(final_features)/len(stage1_features)*100:.1f}%)")
        
        print(f"\nFINAL RESULTS:")
        print(f"- Total features: {n_features} → {len(final_features)} "
              f"({len(final_features)/n_features*100:.1f}%)")
        print(f"- Final fitness: {stage2_fitness:.4f}")
        print(f"- Total evaluations: {len(stage1_progress) + len(stage2_progress)}")
        print("="*60)
    
    # Return comprehensive results
    results = {
        'stage1_weights': stage1_weights,
        'stage1_fitness': stage1_fitness,
        'stage1_progress': stage1_progress,
        'stage1_features': stage1_features,
        'stage2_weights': stage2_weights,
        'stage2_fitness': stage2_fitness,
        'stage2_progress': stage2_progress,
        'final_features': final_features,
        'total_evaluations': len(stage1_progress) + len(stage2_progress)
    }
    
    return results


# Convenience function that mimics your original function interface
def run_two_stage_pso_simple(X, Y, estimator, n_features,
                             stage1_threshold=0.4, stage2_threshold=0.7):
    """
    Simplified two-stage PSO with default parameters
    Returns same format as original function for easy replacement
    """
    results = run_two_stage_pso(
        X, Y, estimator, n_features,
        stage1_threshold=stage1_threshold,
        stage2_threshold=stage2_threshold
    )
    
    # Return in same format as original function
    return (results['stage2_weights'], 
            results['stage2_fitness'], 
            results['stage1_progress'] + results['stage2_progress'],
            results['final_features'])


In [138]:

directory_path = './datasets'
data_health = get_samples(os.path.join(directory_path, 'DT.Healthy.csv'))

# Load the PAN-CANCER-TRANSPOSED.csv data
healthy_cases, prebrca_cases, cancer_cases = split_healthy_data(data_health)

# Combine the data into a single dataframe
# Tag each list of cases
healthy_cases = pd.DataFrame(healthy_cases)
healthy_cases['Tag'] = 'HEALTHY'
prebrca_cases = pd.DataFrame(prebrca_cases)
prebrca_cases['Tag'] = 'PRE-BRCA'
cancer_cases = pd.DataFrame(cancer_cases)
cancer_cases['Tag'] = 'BRCA'

print("Data loaded successfully.")

df_cancer = pd.concat([healthy_cases, prebrca_cases, cancer_cases], ignore_index=True) #blood samples
X = df_cancer.iloc[:, :-1].apply(pd.to_numeric, errors='coerce')
Y = df_cancer.iloc[:, -1]

feature_names = np.array(data_health[0][:-1])

# Fill missing values with the lowest value of its cpg site
X = X.apply(lambda col: col.fillna(col.min()), axis=0)

print(feature_names)  # Display first 5 feature names for brevity

n_features = X.shape[1]
print(f"Loaded dataset with {n_features} features and {len(Y)} samples")

# Use DecisionTreeClassifier as the estimator
estimator = MyXGboost.DecisionTreeMultiClass()


/Users/leandro/master/Mamo.IA/helpers/datasetHelper.py:6: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257

Data loaded successfully.
['cg00000292' 'cg00002426' 'cg00003994' ... 'cg27662379' 'cg27662877'
 'cg27665659']
Loaded dataset with 27578 features and 790 samples


In [139]:

# 2) Run PSO
# For high-dimensional data (27k features), reduce swarmsize and maxiter for tractability
best_weights, best_fitness, progress, X_selected = run_two_stage_pso_simple(
    X, Y, estimator, n_features,
    stage1_threshold=0.4,
    stage2_threshold=0.7
)
print(f"Done PSO → best fitness = {best_fitness:.4f}")

print(X_selected) 
print(best_fitness)
print(best_weights.shape)


X_selected_pso = X.iloc[:, X_selected]

# Show the selected features
print("Selected features:")
selected_feature_names = feature_names[X_selected]
print(selected_feature_names)

STARTING TWO-STAGE PSO FEATURE SELECTION

STAGE 1: Coarse Selection
- Original features: 27578
- Swarm size: 30
- Max iterations: 25
- Threshold: 0.4
shape of X: (790, 27578)
shape of Y: (790,)
----------------------------------------
Eval 10: best fitness so far = 0.5765
Eval 20: best fitness so far = 0.5489
Eval 30: best fitness so far = 0.5489
Eval 40: best fitness so far = 0.5489
Eval 50: best fitness so far = 0.5489
Eval 60: best fitness so far = 0.5489
Eval 70: best fitness so far = 0.5489
Eval 80: best fitness so far = 0.5485
Eval 90: best fitness so far = 0.5485
Eval 100: best fitness so far = 0.5485
Eval 110: best fitness so far = 0.5485
Eval 120: best fitness so far = 0.5485
Eval 130: best fitness so far = 0.5485
Eval 140: best fitness so far = 0.5485
Eval 150: best fitness so far = 0.5485
Eval 160: best fitness so far = 0.5485
Eval 170: best fitness so far = 0.5485
Eval 180: best fitness so far = 0.5485
Eval 190: best fitness so far = 0.5485
Eval 200: best fitness so far = 0

In [140]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, cohen_kappa_score
from deap import base, creator, tools, algorithms
import random

def run_ga_with_progress(X, Y, estimator, n_features,
                        pop_size=50, n_generations=100, threshold=0.7,
                        cx_prob=0.5, mut_prob=0.2, tournament_size=3):
   
    progress = []
    
    # Create a scorer for Cohen's Kappa
    kappa_scorer = make_scorer(cohen_kappa_score)
    
    # DEAP setup
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    toolbox = base.Toolbox()
    
    # Gene initialization (random float between 0 and 1)
    toolbox.register("attr_float", random.random)
    
    # Individual initialization
    toolbox.register("individual", tools.initRepeat, creator.Individual,
                    toolbox.attr_float, n_features)
    
    # Population initialization
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    def evaluate_individual(individual):
        """Evaluate an individual using cross-validation with Cohen's Kappa"""
        weights = np.array(individual)
        Xw = X * weights
        
        # Use cross-validation with Cohen's Kappa
        kappa_scores = cross_val_score(estimator, Xw, Y, cv=5, scoring=kappa_scorer)
        fitness = 1 - kappa_scores.mean()  # Minimize (1 - kappa)
        
        progress.append(fitness)
        
        if len(progress) % 10 == 0:
            print(f"Eval {len(progress)}: best fitness so far = {min(progress):.4f}")
        
        return (fitness,)
    
    # Register genetic operators
    toolbox.register("evaluate", evaluate_individual)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=tournament_size)
    
    # Create initial population
    pop = toolbox.population(n=pop_size)
    
    # Evaluate initial population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    # Evolution loop
    for generation in range(n_generations):
        print(f"Generation {generation + 1}/{n_generations}")
        
        # Select parents
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))
        
        # Apply crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cx_prob:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        # Apply mutation
        for mutant in offspring:
            if random.random() < mut_prob:
                toolbox.mutate(mutant)
                # Clip values to [0, 1] range
                for i in range(len(mutant)):
                    mutant[i] = max(0, min(1, mutant[i]))
                del mutant.fitness.values
        
        # Evaluate offspring with invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
    # Find best individual
    best_individual = tools.selBest(pop, 1)[0]
    best_fitness = best_individual.fitness.values[0]
    
    # Select features based on threshold
    mask = np.array(best_individual) > threshold
    selected_features = np.where(mask)[0].tolist()
    
    # Clean up DEAP classes to avoid conflicts in future runs
    del creator.FitnessMin
    del creator.Individual
    
    return list(best_individual), best_fitness, progress, selected_features


In [141]:
best_weights_ga, best_fitness_ga, progress_ga, X_selected_ga = run_ga_with_progress(
    X, Y, estimator, X.shape[1], 
    pop_size=30, n_generations=500, threshold=0.5
)
print(f"Done GA → best fitness = {best_fitness_ga:.4f}")
print(f"Number of selected features: {len(selected_feature_names)}")
print(f"Selected feature indices: {selected_feature_names[:10]}...")  # Show first 10

# Convert best_weights_ga to numpy array before comparison
best_weights_ga_array = np.array(best_weights_ga)
mask_ga = best_weights_ga_array > 0.9
X_selected_ga = X.iloc[:, mask_ga]

selected_feature_names_ga = feature_names[mask_ga]
print(selected_feature_names_ga)

Eval 10: best fitness so far = 0.5907
Eval 20: best fitness so far = 0.5862
Eval 30: best fitness so far = 0.5862
Generation 1/500
Eval 40: best fitness so far = 0.5862
Eval 50: best fitness so far = 0.5518
Generation 2/500
Eval 60: best fitness so far = 0.5518
Eval 70: best fitness so far = 0.5518
Generation 3/500
Eval 80: best fitness so far = 0.5518
Eval 90: best fitness so far = 0.5306
Generation 4/500
Eval 100: best fitness so far = 0.5306
Generation 5/500
Eval 110: best fitness so far = 0.5306
Eval 120: best fitness so far = 0.5306
Generation 6/500
Eval 130: best fitness so far = 0.5306
Eval 140: best fitness so far = 0.5306
Generation 7/500
Eval 150: best fitness so far = 0.5306
Eval 160: best fitness so far = 0.5306
Generation 8/500
Eval 170: best fitness so far = 0.5306
Eval 180: best fitness so far = 0.5306
Generation 9/500
Eval 190: best fitness so far = 0.5306
Generation 10/500
Eval 200: best fitness so far = 0.5306
Generation 11/500
Eval 210: best fitness so far = 0.5306
E

In [143]:
# Use LabelEncoder to encode the target classes
label_encoder = LabelEncoder()
# Define the order: Healthy -> Pre-BRCA -> BRCA
ordered_labels = ['HEALTHY', 'PRE-BRCA', 'BRCA']
label_encoder.fit(ordered_labels)
Y_encoded = label_encoder.transform(Y)

print(f"Encoded target classes: {label_encoder.classes_}")

# 1) evaluate with all features
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(
    X, Y_encoded, test_size=0.2, random_state=42
)
# Apply SMOTE to balance the training instances
smote = SMOTE(sampling_strategy='auto', random_state=None, k_neighbors=10)
X_train_all, y_train_all = smote.fit_resample(X_train_all, y_train_all)

# 2) evaluate with selected features
X_train_ga, X_test_ga, y_train_ga, y_test_ga = train_test_split(
    X_selected_ga, Y_encoded, test_size=0.2, random_state=42
)
# Apply SMOTE to balance the training instances
smote = SMOTE(sampling_strategy='auto', random_state=None, k_neighbors=10)
X_train_ga, y_train_ga = smote.fit_resample(X_train_ga, y_train_ga)

# 3) evaluate with PSO selected features
X_train_pso, X_test_pso, y_train_pso, y_test_pso = train_test_split(
    X_selected_pso, Y_encoded, test_size=0.2, random_state=42
)
# Apply SMOTE to balance the training instances
smote = SMOTE(sampling_strategy='auto', random_state=None, k_neighbors=10)
X_train_pso, y_train_pso = smote.fit_resample(X_train_pso, y_train_pso)

Encoded target classes: ['BRCA' 'HEALTHY' 'PRE-BRCA']


/Users/leandro/master/Mamo.IA/local-env/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/leandro/master/Mamo.IA/local-env/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/leandro/master/Mamo.IA/local-env/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [144]:
modes = [
    {
        'Name': 'RandomForest100',
        'Model': MyXGboost.RandomForest100()
    },
    {
        'Name': 'RandomForest200',
        'Model': MyXGboost.RandomForest200()
    },
    {
        'Name': 'RandomForest300',
        'Model': MyXGboost.RandomForest300()
    },
    {
        'Name': 'XGBoost',
        'Model': MyXGboost.XGBoostMultiClass()
    },
    {
        'Name': 'LightGBM',
        'Model': MyXGboost.LightGBMMulticlass()
    },
    {
        'Name': 'GradientBoosting',
        'Model': MyXGboost.GradientBoosting()
    }
]

In [150]:
# Print dimensions of different training datasets
print("Training data dimensions:")
print(f"Original data (X_train_all): {X_train_all.shape}")
print(f"GA selected features (X_train_ga): {X_train_ga.shape}")
print(f"PSO selected features (X_train_pso): {X_train_pso.shape}")

for feature_set in [('GA', X_train_ga, X_test_ga, y_train_ga, y_test_ga), 
                   ('PSO', X_train_pso, X_test_pso, y_train_pso, y_test_pso)]:
    method, X_train, X_test, y_train, y_test = feature_set
    print(f"\n=== Results for {method} selected features ===")
    
    for m in modes:
        # Split training data into train and validation sets for early stopping
        X_train_split, X_val, y_train_split, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42
        )
        
        if m['Name'] == 'XGBoost':
            # For XGBoost, use validation set for early stopping
            selector = m['Model'].fit(
                X_train_split, y_train_split,
                eval_set=[(X_val, y_val)],
                verbose=True
            )
        else:
            # For other models, use all training data
            selector = m['Model'].fit(X_train, y_train)
        
        # Evaluate the model
        y_pred = selector.predict(X_test)
        y_pred_proba = selector.predict_proba(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        # Handle binary and multiclass cases for ROC AUC
        if y_pred_proba.shape[1] == 2:
            roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])
        else:
            roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')

        print(f"\nModel: {m['Name']}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"ROC AUC: {roc_auc:.4f}")

        # Compute Kappa index
        kappa = cohen_kappa_score(y_test, y_pred)
        print(f"Kappa index: {kappa:.4f}")

        # Confusion matrix
        display_confusion_matrix_pink_variants(selector, X_test, y_test)
        
        # Feature importance (if available)
        if hasattr(selector, 'feature_importances_'):
            importances = selector.feature_importances_
            indices = np.argsort(importances)[::-1][:20]
            print("\nTop 20 important features:")
            feature_names = selected_feature_names_ga if method == 'GA' else selected_feature_names
            for rank, idx in enumerate(indices, 1):
                print(f"{rank}. {feature_names[idx]}: {importances[idx]:.4f}")
        else:
            print("\nThis model does not provide feature importances.")
        print("-" * 80)

Training data dimensions:
Original data (X_train_all): (1425, 27578)
GA selected features (X_train_ga): (1425, 3295)
PSO selected features (X_train_pso): (1425, 4890)

=== Results for GA selected features ===

Model: RandomForest100
Accuracy: 0.7658
F1 Score: 0.7777
ROC AUC: 0.9114
Kappa index: 0.5363

Top 20 important features:
1. cg15679095: 0.0189
2. cg19233923: 0.0171
3. cg21727574: 0.0148
4. cg26538116: 0.0132
5. cg25658980: 0.0118
6. cg15888522: 0.0114
7. cg15869642: 0.0114
8. cg13006591: 0.0113
9. cg21642076: 0.0111
10. cg15264273: 0.0108
11. cg23887102: 0.0098
12. cg13917394: 0.0089
13. cg19815589: 0.0082
14. cg26478992: 0.0077
15. cg26117431: 0.0077
16. cg11116440: 0.0076
17. cg04099420: 0.0076
18. cg18098286: 0.0076
19. cg00519208: 0.0074
20. cg25994725: 0.0073
--------------------------------------------------------------------------------

Model: RandomForest200
Accuracy: 0.7342
F1 Score: 0.7466
ROC AUC: 0.8968
Kappa index: 0.4534

Top 20 important features:
1. cg04837997: 

/Users/leandro/master/Mamo.IA/local-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[09:38:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




[2]	validation_0-mlogloss:0.95812
[3]	validation_0-mlogloss:0.91790
[4]	validation_0-mlogloss:0.88205
[5]	validation_0-mlogloss:0.84828
[6]	validation_0-mlogloss:0.81680
[7]	validation_0-mlogloss:0.78802
[8]	validation_0-mlogloss:0.76132
[9]	validation_0-mlogloss:0.73499
[10]	validation_0-mlogloss:0.71030
[11]	validation_0-mlogloss:0.68840
[12]	validation_0-mlogloss:0.66751
[13]	validation_0-mlogloss:0.64825
[14]	validation_0-mlogloss:0.62950
[15]	validation_0-mlogloss:0.61115
[16]	validation_0-mlogloss:0.59613
[17]	validation_0-mlogloss:0.57987
[18]	validation_0-mlogloss:0.56438
[19]	validation_0-mlogloss:0.54873
[20]	validation_0-mlogloss:0.53491
[21]	validation_0-mlogloss:0.52279
[22]	validation_0-mlogloss:0.51035
[23]	validation_0-mlogloss:0.49899
[24]	validation_0-mlogloss:0.48864
[25]	validation_0-mlogloss:0.47833
[26]	validation_0-mlogloss:0.46775
[27]	validation_0-mlogloss:0.45887
[28]	validation_0-mlogloss:0.45065
[29]	validation_0-mlogloss:0.44222
[30]	validation_0-mlogloss:0

/Users/leandro/master/Mamo.IA/local-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[09:42:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




[1]	validation_0-mlogloss:1.00231
[2]	validation_0-mlogloss:0.95936
[3]	validation_0-mlogloss:0.92317
[4]	validation_0-mlogloss:0.88363
[5]	validation_0-mlogloss:0.84866
[6]	validation_0-mlogloss:0.81641
[7]	validation_0-mlogloss:0.78746
[8]	validation_0-mlogloss:0.75816
[9]	validation_0-mlogloss:0.73076
[10]	validation_0-mlogloss:0.70663
[11]	validation_0-mlogloss:0.68159
[12]	validation_0-mlogloss:0.65848
[13]	validation_0-mlogloss:0.63613
[14]	validation_0-mlogloss:0.61655
[15]	validation_0-mlogloss:0.59871
[16]	validation_0-mlogloss:0.58328
[17]	validation_0-mlogloss:0.56798
[18]	validation_0-mlogloss:0.55160
[19]	validation_0-mlogloss:0.53622
[20]	validation_0-mlogloss:0.52279
[21]	validation_0-mlogloss:0.50876
[22]	validation_0-mlogloss:0.49660
[23]	validation_0-mlogloss:0.48499
[24]	validation_0-mlogloss:0.47348
[25]	validation_0-mlogloss:0.46265
[26]	validation_0-mlogloss:0.45221
[27]	validation_0-mlogloss:0.44136
[28]	validation_0-mlogloss:0.43273
[29]	validation_0-mlogloss:0.